In [9]:
from typing import Annotated
from dotenv import load_dotenv

load_dotenv()

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [14]:
from langchain.chat_models import init_chat_model

In [17]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages":[llm.invoke(state["messages"])]}

builder =  StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [18]:
messages = {"role": "user", "content": "What were the names of the people in the Apollo 13 mission?"}
response = graph.invoke({"messages":[messages]})

response["messages"]

[HumanMessage(content='What were the names of the people in the Apollo 13 mission?', additional_kwargs={}, response_metadata={}, id='c544eb5c-494e-4673-983a-2a67be43c561'),
 AIMessage(content='The crew of the Apollo 13 mission were:\n\n*   **James A. Lovell Jr.** (Commander)\n*   **John L. "Jack" Swigert Jr.** (Command Module Pilot)\n*   **Fred W. Haise Jr.** (Lunar Module Pilot)', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1c67e80e-fc91-4052-ad2d-f000f543186e-0', usage_metadata={'input_tokens': 15, 'output_tokens': 59, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})]

In [21]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: This is a trick question! The first humans evolved and lived on Earth, so there was no "first man to walk on Earth" in the sense of arriving from somewhere else. Walking is how humans naturally move.


KeyboardInterrupt: Interrupted by user